In [63]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder , OneHotEncoder
import pickle

In [64]:
## load the trained model, scaler pickle , onehot
model = load_model('model.h5') 

## onehot encoder , label encoder , scaler 
with open ('label_encoder_gender.pkl','rb') as file :
    label_encoder_gender =pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [65]:
# input data 
input_data  = {
    'CreditScore' : 300, 
    'Geography' : 'France',
    'Gender' : 'Male', 
    'Age': 15, 
    'Tenure' : 3, 
    'Balance' : 160, 
    'NumOfProducts' : 2, 
    'HasCrCard' : 1,
    'IsActiveMember' : 1, 
    'EstimatedSalary' : 1500
}

In [66]:
## create a input DataFrame
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,300,France,Male,15,3,160,2,1,1,1500


In [67]:
## Now onehotencoder for Georaphy 
geo_encoder = onehot_encoder_geo.transform(input_df[['Geography']])

# to check the new columns name after onehot encoder  
geo_encoder_columns = onehot_encoder_geo.get_feature_names_out(['Geography'])

# First we need to convert the geo_encoder to array then we will abale to create a dataframe. 
geo_encoder_array = geo_encoder.toarray()

geo_encoder_df = pd.DataFrame (geo_encoder_array, columns=geo_encoder_columns)
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [68]:
## Lebal encoder for gender 
input_df['Gender'] = label_encoder_gender.fit_transform(input_df['Gender'])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,300,France,0,15,3,160,2,1,1,1500


In [69]:
## Combine one hot encoder columns with the original data
input_data= pd.concat([input_df.drop('Geography', axis=1), geo_encoder_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,300,0,15,3,160,2,1,1,1500,1.0,0.0,0.0


In [70]:
## Scalling the input data
input_data_scaler= scaler.transform(input_data)
input_data_scaler

array([[-3.54326899, -1.09997489, -2.31795024, -0.69844549, -1.25342145,
         0.80510537,  0.63367318,  0.95214374, -1.69216683,  0.98019606,
        -0.57581067, -0.56349184]])

In [71]:
#Predict churn 
prediction = model.predict(input_data_scaler)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[0.00510889]], dtype=float32)

In [72]:
if prediction[0][0] >= 0.05:
    print("This customer is likely to churn")
    print(f"And the possibility is {round(prediction[0][0] *100,2)} %")
else:
    print("This customer is not likely to churn")
    print(f"And the possibility is {round(prediction[0][0] *100,2)} %")

This customer is not likely to churn
And the possibility is 0.51 %
